In [1]:
import os
import tensorflow as tf
import numpy as np 
from PIL import Image

In [2]:
# USE GPU to run

print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
True


In [80]:
# Load data

def load_data():
    cur_path = os.getcwd()+'\\'
    classes = [x for x in os.listdir(cur_path) if not x.startswith('.')][:10]
    label_set = set()
    data = []
    labels = []
    for i in classes:
        if not (i.replace(cur_path, "")).startswith('.'):
            images = os.listdir(i)
            for a in images:
                if a.endswith('.jpg'):
                    image = Image.open(i + '\\' + a)
                    image = image.resize((60, 60))
                        
                    image = np.asarray(image)
                        
                    data.append(image)
                        
                    labels.append(classes.index(i.split('\\')[-1]))
                    label_set.add(i.split('\\')[-1])

    return data, labels, len(label_set) 

data, labels, num_len = load_data()

In [81]:
print("Amount of images: ", len(data))
print("Dimension of each image: ", data[0].shape)
print("Num length: ", num_len)


Amount of images:  1524
Dimension of each image:  (60, 60, 3)
Num length:  10


In [84]:
data = [i for i in data if i.shape == (60, 60, 3)]
data = np.array(data)
data = data.astype('float32')/255.0

In [85]:
labels = tf.one_hot(tf.cast(labels, dtype=tf.int32), num_len)
labels

<tf.Tensor: shape=(1524, 10, 10), dtype=float32, numpy=
array([[[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0.